In [69]:
WINDOW_LENGTHS = [10, 25, 50, 100]
DO_BLUR = False
BLACK_THRESH = 10  # x <= threshhold
WHITE_THRESH = 240 # x > thresh

In [70]:
import numpy as np
import matplotlib.pyplot as plt

import os
import sys

In [71]:
sys.path.append(os.path.abspath('..'))
import live_3d_rendering as l3d

In [72]:
import glob
import itertools as it
from typing import Iterator, Optional

celiacs_dir = "images/mar29-higher-res/celiac/"
controls_dir = "images/mar29-higher-res/control/"
img_file_extensions = ['jpg', 'jpeg', 'png', 'JPG', 'JPEG', 'PNG']

def file_iterator(directory, file_ext) -> Iterator[str]:
    return glob.glob(os.path.join(directory, f"*.{file_ext}"), recursive=False)

celiac_files = it.chain(*[file_iterator(celiacs_dir, ext) for ext in img_file_extensions])
control_files = it.chain(*[file_iterator(controls_dir, ext) for ext in img_file_extensions])

In [73]:
from PIL import Image, ImageFilter

if DO_BLUR:
    celiac_ims = [l3d.preprocess(Image.open(f)).filter(ImageFilter.GaussianBlur(5)) for f in celiac_files]
    control_ims = [l3d.preprocess(Image.open(f)).filter(ImageFilter.GaussianBlur(5)) for f in control_files]
else:
    celiac_ims = [l3d.preprocess(Image.open(f), white_thresh=WHITE_THRESH) for f in celiac_files]
    control_ims = [l3d.preprocess(Image.open(f), white_thresh=WHITE_THRESH) for f in control_files]
    

In [74]:
from live_3d_rendering import local_stats as stats

def window_ok(w: stats.Window, black_thresh: int) -> bool:
    """Return True if w is nonempty and at most 1% pixels <= black_thresh."""
    return w.access().size > 0 and np.sum(w.access() <= black_thresh) < 0.01 * w.access().size

In [75]:
# def window_iterator(im: Image.Image, side_len: int, black_thresh: Optional[int] = 10) -> Iterator[stats.Window]:
#     arr = np.array(im)
#     yield stats.Window(arr, 0, 0, *arr.shape)

def window_iterator(im: Image.Image, side_len: int, black_thresh: Optional[int] = 10) -> Iterator[stats.Window]:
    arr = np.array(im)
    for row in range(0, max(1, im.size[0] - side_len), side_len):
        for col in range(0, max(1, im.size[1] - side_len), side_len):
            window = stats.Window(arr, row, col, side_len, side_len)
            if window_ok(window, black_thresh):
                yield window

In [76]:
from live_3d_rendering.stats_functions import (
    local_min,
    local_max,
    local_mean,
    local_variance,
)
import pandas as pd

funcs = [local_min, local_max, local_mean, local_variance, lambda x: np.sqrt(x.var())]
func_names = ['min', 'max', 'mean', 'variance', 'std']

In [77]:
celiac_info_by_wlen = {wlen: [] for wlen in WINDOW_LENGTHS}
control_info_by_wlen = {wlen: [] for wlen in WINDOW_LENGTHS}

for window_len in WINDOW_LENGTHS:
    
    for im in celiac_ims:
        for window in window_iterator(im, window_len, BLACK_THRESH):
            celiac_info_by_wlen[window_len].append(window.get_stats(funcs))
    
    for im in control_ims:
        for window in window_iterator(im, window_len, BLACK_THRESH):
            control_info_by_wlen[window_len].append(window.get_stats(funcs))

In [78]:
for wlen in WINDOW_LENGTHS:
    celiac_df = pd.DataFrame(celiac_info_by_wlen[wlen], columns=func_names)
    celiac_df.to_csv(f"celiac_{wlen}_by_{wlen}.csv", index=False)
    control_df = pd.DataFrame(control_info_by_wlen[wlen], columns=func_names)
    control_df.to_csv(f"control_{wlen}_by_{wlen}.csv", index=False)

# Visualization

In [79]:
def stats_of_wlen(WL):
    celiac = pd.read_csv(f"celiac_{WL}_by_{WL}.csv")
    celiac_means = celiac.mean()
    celiac_stds = celiac.std()
    control = pd.read_csv(f"control_{WL}_by_{WL}.csv")
    control_means = control.mean()
    control_stds = control.std()
    return pd.DataFrame(
        [celiac_means, celiac_stds, control_means, control_stds],
        columns=['min', 'max', 'mean', 'variance', 'std'],
        index=["celiac (mean value)", "celiac (std)", "control (mean value)", "control (std)"],
    )
from IPython.display import display
print('---------------------------------------------------\n')
for wlen in WINDOW_LENGTHS:
    print(f"{wlen}x{wlen} windows")
    display(stats_of_wlen(wlen))
    print('---------------------------------------------------\n')

---------------------------------------------------

10x10 windows


,min,max,mean,variance,std
celiac (mean value),78.911117,94.109683,86.685100,27.533552,3.825632
celiac (std),33.786935,35.026077,34.085710,84.946033,3.591435
control (mean value),88.143702,100.494016,94.466167,18.251203,3.089696
control (std),26.707061,25.646540,25.777523,60.525676,2.950455


---------------------------------------------------

25x25 windows


,min,max,mean,variance,std
celiac (mean value),71.906362,103.292822,88.772770,75.391939,6.870203
celiac (std),32.774325,34.776445,32.722492,162.456398,5.310070
control (mean value),81.482227,107.339436,95.430179,52.911532,5.733548
control (std),27.483378,25.374515,24.960065,108.384691,4.476720


---------------------------------------------------

50x50 windows


,min,max,mean,variance,std
celiac (mean value),62.535050,114.447783,91.156072,145.921412,10.204836
celiac (std),31.430392,34.504216,31.107038,217.490567,6.466269
control (mean value),72.692561,115.845951,96.878070,105.889538,8.707758
control (std),27.625548,24.902973,23.568234,151.520846,5.484916


---------------------------------------------------

100x100 windows


,min,max,mean,variance,std
celiac (mean value),50.240143,133.978495,95.153362,291.175784,15.423218
celiac (std),28.535533,32.001342,28.890444,286.359910,7.313813
control (mean value),60.034921,130.073016,99.566945,209.884028,13.107097
control (std),25.609924,23.040211,21.220269,208.608126,6.181369


---------------------------------------------------



### (ignore the rest of this document, it's scratch work)